In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/noice-audio/705_AUDIO.wav
/kaggle/input/noice-audio/679_AUDIO.wav


In [2]:
!pip install git+https://github.com/openai/whisper.git 

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-81aesdvb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-81aesdvb
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=ffa38663b7a9fd196421f13c97cd9083b2f4129fa0d5aba69c7264ccf02b1cc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-_pnzk40h/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [3]:
import librosa
import numpy as np
import soundfile as sf

def remove_noise_and_silence(audio_file, output_file, top_db=30):
    y, sr = librosa.load(audio_file, sr=None)
    non_silent_intervals = librosa.effects.split(y, top_db=top_db)
    non_silent_audio = np.concatenate([y[start:end] for start, end in non_silent_intervals])
    sf.write(output_file, non_silent_audio, sr)
    print(f'Noise and silence removed from {audio_file}, saved to {output_file}')
    

input_audio = '/kaggle/input/noice-audio/679_AUDIO.wav'  # Replace with your input file
output_audio = 'noise_removed_audio.wav'
remove_noise_and_silence(input_audio, output_audio)


Noise and silence removed from /kaggle/input/noice-audio/679_AUDIO.wav, saved to noise_removed_audio.wav


In [4]:
import whisper

# Load Whisper model
whisper_model = whisper.load_model('base')

def transcribe_audio(audio_file):
    # Transcribe the audio
    result = whisper_model.transcribe(audio_file, fp16=False)
    transcription = result['text']
    print(f'Transcription: {transcription}')
    return transcription

# Example usage
transcribed_text = transcribe_audio(output_audio)


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 127MiB/s]
/opt/conda/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription:  Yeah, I'm a rumb and I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, but I live in the road long, But I live

In [5]:
import torch

In [7]:
from transformers import BertForSequenceClassification, BertTokenizer
model = BertForSequenceClassification.from_pretrained("Thiru-KTM/audio-text-bert")
tokenizer = BertTokenizer.from_pretrained("Thiru-KTM/audio-text-bert")

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [8]:
def preprocess_for_prediction(text, tokenizer, max_length=512):
    # Tokenize the user input
    inputs = tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        padding='max_length',  # Ensures the input is padded to max_length
        return_tensors="pt"    # Return as PyTorch tensors
    )
    return inputs


In [9]:
def predict(text, model, tokenizer):
    # Preprocess the input
    inputs = preprocess_for_prediction(text, tokenizer)
    
    # Move inputs to the device (CPU or GPU)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Make sure the model is in evaluation mode
    model.eval()

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the predicted class (logits -> argmax)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class


In [10]:
#user_input = "This is an example of text to classify."

# Get the predicted label
predicted_label = predict(transcribed_text, model, tokenizer)

print(f"Predicted label: {predicted_label}")

Predicted label: 1
